# Bot xadrez nível básico

In [ ]:
import chess
import random
import chess.polyglot
from IPython.display import clear_output
from copy import deepcopy

In [ ]:
def definirLado():
    lado = input('Defina o seu lado (W/B): ').upper()
    if lado == 'W':
        return chess.WHITE
    elif lado == 'B':
        return chess.BLACK
    else:
        print('Lado inválido. Escolha W para branco ou B para preto.')
        return definirLado()

In [ ]:
board = chess.Board()
lado_escolhido = definirLado()

In [ ]:
def isEmpate(board=board):
    if board.is_stalemate() or board.is_fivefold_repetition() or board.is_seventyfive_moves():
        return True
    else:
        return False

In [ ]:
def jogadaOponente(board):
    while True:
        resposta = input('Informe a jogada do oponente (use a notação de casas): ')
        if len(resposta) == 4  and resposta.isalnum():
            jogada = chess.Move.from_uci(resposta)
            if jogada in board.legal_moves:
                board.push(jogada)
                break
            else:
                print("Jogada ilegal. Tente novamente.")
        else:
            print("Jogada inválida. A notação UCI deve ter 4 ou 5 caracteres.")

In [ ]:
def oponenteTeste(board):
    return random.choice(list(board.legal_moves))

In [ ]:
reader = chess.polyglot.open_reader('Human.bin')

pecas_valores= {'p': -10,
          'n': -30,
          'b': -30,
          'r': -50,
          'q': -90,
          'k': -900,
          'P': 10,
          'N': 30,
          'B': 30,
          'R': 50,
          'Q': 90,
          'K': 900,
          }

def eval_board(board):
    score = 0
    pieces = board.piece_map()
    for key in pieces:
        score += pecas_valores[str(pieces[key])]

    return score

def eval_space(board):
    no_moves = len(list(board.legal_moves))
    value = (no_moves/(20+no_moves))
    
    if board.turn == True:
        return value
    else:
        return -value

def minMax(board, profundidade):

    opening_move = reader.get(board)
    if opening_move == None:
        pass
    else:
        return opening_move.move

    moves = list(board.legal_moves)
    scores = []

    #score each move
    for move in moves:
        #temp allows us to leave the original game state unchanged
        temp = deepcopy(board)
        temp.push(move)

        #here we must check that the game is not over
        outcome = temp.outcome()
        
        #if checkmate
        if outcome == None:
            #if we have not got to the final depth
            #we search more moves ahead
            if profundidade > 1:
                temp_best_move = minMax(temp, profundidade - 1)
                temp.push(temp_best_move)

            scores.append(eval_board(temp))

        #if checkmate
        elif temp.is_checkmate():

            # we return this as best move as it is checkmate
            return move

        # if stalemate
        else:
            #value to disencourage a draw
            #the higher the less likely to draw
            #default value should be 0
            #we often pick 0.1 to get the bot out of loops in bot vs bot
            val = 1000
            if board.turn == True:
                scores.append(-val)
            else:
                scores.append(val)

        #this is the secondary eval function
        scores[-1] = scores[-1] + eval_space(temp)

    if board.turn == True:
        best_move = moves[scores.index(max(scores))]
    else:
        best_move = moves[scores.index(min(scores))]

    return best_move        

In [ ]:
def jogar(board):
    jogada_bot = minMax(board, profundidade = 3)
    if jogada_bot:
        board.push(jogada_bot)
    else:
        print("Não é a vez do bot jogar.")

In [ ]:
def main(board, lado_escolhido):
    while not board.is_game_over():
        if board.turn == lado_escolhido:
            jogar(board)
        else:
            jogadaOponente(board)
        clear_output()
        display(board)

    if board.is_checkmate():
        print("Xeque-mate! O jogo acabou.")
    elif isEmpate(board):
        print("Empate! O jogo terminou em empate.")
    else:
        print("O jogo terminou por algum motivo desconhecido.")

In [ ]:
main(board, lado_escolhido)